![](https://images.pexels.com/photos/3738673/pexels-photo-3738673.jpeg?auto=compress&cs=tinysrgb&h=750&w=1260)

# OpenCV Trackers

Kevin J Walchko, Phd

2 Nov 2017

---

OpenCV comes with several tracking algorithms. In this lesson we are going to cover the basica ideas of how they work and do a couple simple examples to show you they work. Later on, you can delve deeper into these trackers and understand the mathematics of how they work and even adapt them for more complex situations. But today, we are just going to introduce them and explain the basics.

Now, OpenCV offers the Kalman filter too, which is a very powerful filter that can be used for tracking and estimating. However, it is not simple to setup and I ran out of time trying to get the examples together. 

## References

- [OpenCV trackers](https://www.learnopencv.com/object-tracking-using-opencv-cpp-python/)
- [pyimagesearch.com overview](https://www.pyimagesearch.com/2018/07/30/opencv-object-tracking/)

## Setup

In [6]:
%matplotlib inline 

#from __future__ import print_function
# these imports let you use opencv
import cv2          # opencv itself
import numpy as np  # matrix manipulations
# to play an mpeg4 video, you can do this:
import IPython
from IPython.display import HTML # need this for embedding a movie in an iframe
from matplotlib import pyplot as plt           
import pylab                                   
pylab.rcParams['figure.figsize'] = (10.0, 8.0) 

In [3]:
# simple function to save a video
import platform
def videoWrite(frames, fname='out.mp4'):
    frame_height, frame_width, _ = frames[0].shape
    
    # pick a good encoder for the current OS
    sys = platform.system()
    if sys in ['Darwin']:
        fourcc = 'avc1'
    else:
        fourcc = 'mjpg'
        
    out = cv2.VideoWriter(
        fname,
        cv2.VideoWriter_fourcc(*fourcc), 
        30, 
        (frame_width,frame_height))
    for frame in frames:
        out.write(frame)
    out.release()

# Object Tracking Algorithms

In this section, we will dig a bit into different tracking algorithms. The goal is not to have a deep theoretical understanding of every tracker, but to understand them from a practical standpoint.

Let me begin by first explaining some general principles behind tracking. In tracking, our goal is to find an object in the current frame. We assume we have tracked the object successfully in all ( or nearly all ) previous frames. Since we have tracked the object up until the current frame, we know how it has been moving. In other words, we know the parameters of the motion model. The motion model is just a fancy way of saying that you know the location and the velocity ( speed + direction of motion ) in 2D of the object in the previous frames. If you knew nothing else about the object, you could predict the new location based on the current motion model.

But we have more information that just the motion of the object. We know how the object looks in each of the previous frames. In other words, we can build an appearance model that encodes what the object looks like. This appearance model can be used to search in a small neighborhood of the location predicted by the motion model to more accurately predict the location of the object.

- The **motion model** predicts the approximate location of the object. 
- The **appearance model** fine tunes this estimate to provide a more accurate estimate based on appearance.

If the object was very simple and did not change it’s appearance much, we could use a simple template as an appearance model and look for that template. However, real life is not that simple. The appearance of an object can change dramatically. To tackle this problem, in many modern trackers, this appearance model is a classifier that is trained in an online manner. Don’t panic! Let me explain in simpler terms.

The job of the classifier is to classify a rectangular region of an image as either an object or background. The classifier takes in an image patch as input and returns a score between 0 and 1 to indicate the probability that the image patch contains the object. The score is 0 when it is absolutely sure the image patch is the background and 1 when it is absolutely sure the patch is the object.

In machine learning, we use the word “online” to refer to algorithms that are trained on the fly at run time. An offline classifier may need thousands of examples to train a classifier, but an online classifier is typically trained using a very few examples at run time. Offline classifiers tend to have better performance.

A classifier is trained by feeding it positive ( object ) and negative ( background ) examples. If you want to build a classifier for detecting cats, you train it with thousands of images containing cats and thousands of images that do not contain cats. This way the classifier learns to differentiate what is a cat and what is not. While building an online classifier, we do not have the luxury of having thousands of examples of the positive and negative classes.

Let’s look how different tracking algorithms approach this problem of online training.

## BOOSTING Tracker

This tracker is based on an online version of AdaBoost — the algorithm that the HAAR cascade based face detector uses internally. This classifier needs to be trained at runtime with positive and negative examples of the object. The initial bounding box supplied by the user ( or by another object detection algorithm ) is taken as the positive example for the object, and many image patches outside the bounding box are treated as the background. Given a new frame, the classifier is run on every pixel in the neighborhood of the previous location and the score of the classifier is recorded. The new location of the object is the one where the score is maximum. So now we have one more positive example for the classifier. As more frames come in, the classifier is updated with this additional data.

- **Pros:** None. This algorithm is a decade old and works ok, but I could not find a good reason to use it especially when other advanced trackers (MIL, KCF) based on similar principles are available.

- **Cons:** Tracking performance is mediocre. It does not reliably know when tracking has failed.

## Multiple Instance Learning ( MIL ) Tracker

This tracker is similar in idea to the BOOSTING tracker described above. The big difference is that instead of considering only the current location of the object as a positive example, it looks in a small neighborhood around the current location to generate several potential positive examples. You may be thinking that it is a bad idea because in most of these “positive” examples the object is not centered.

This is where MIL comes to rescue. In MIL, you do not specify positive and negative examples, but positive and negative “bags”. The collection of images in the positive bag are not all positive examples. Instead, only one image in the positive bag needs to be a positive example! In our example, a positive bag contains the patch centered on the current location of the object and also patches in a small neighborhood around it. Even if the current location of the tracked object is not accurate, when samples from the neighborhood of the current location are put in the positive bag, there is a good chance that this bag contains at least one image in which the object is nicely centered. MIL project page has more information for people who like to dig deeper into the inner workings of the MIL tracker.

- **Pros:** The performance is pretty good. It does not drift as much as the BOOSTING tracker and it does a reasonable job under partial occlusion. If you are using OpenCV 3.0, this might be the best tracker available to you. But if you are using a higher version, consider KCF.
- **Cons:** Tracking failure is not reported reliably. Does not recover from full occlusion.

## KCF Tracker

KFC stands for Kernelized Correlation Filters. This tracker builds on the ideas presented in the previous two trackers. This tracker utilizes that fact that the multiple positive samples used in the MIL tracker have large overlapping regions. This overlapping data leads to some nice mathematical properties that is exploited by this tracker to make tracking faster and more accurate at the same time.

- **Pros:** Accuracy and speed are both better than MIL and it reports tracking failure better than BOOSTING and MIL. If you are using OpenCV 3.1 and above, I recommend using this for most applications.
- **Cons:** Does not recover from full occlusion. Not implemented in OpenCV 3.0.

## Tracking, Learning and Detection (TLD) Tracker

As the name suggests, this tracker decomposes the long term tracking task into three components — (short term) tracking, learning, and detection. The tracker follows the object from frame to frame. The detector localizes all appearances that have been observed so far and corrects the tracker if necessary. The learning estimates detector’s errors and updates it to avoid these errors in the future. This output of this tracker tends to jump around a bit. For example, if you are tracking a pedestrian and there are other pedestrians in the scene, this tracker can sometimes temporarily track a different pedestrian than the one you intended to track. On the positive side, this track appears to track an object over a larger scale, motion, and occlusion. If you have a video sequence where the object is hidden behind another object, this tracker may be a good choice.

- **Pros:** Works the best under occlusion over multiple frames. Also, tracks best over scale changes.
- **Cons:** Lots of false positives making it almost unusable.

## MEDIANFLOW Tracker

Internally, this tracker tracks the object in both forward and backward directions in time and measures the discrepancies between these two trajectories. Minimizing this ForwardBackward error enables them to reliably detect tracking failures and select reliable trajectories in video sequences.

In my tests, I found this tracker works best when the motion is predictable and small. Unlike, other trackers that keep going even when the tracking has clearly failed, this tracker knows when the tracking has failed.

- **Pros:** Excellent tracking failure reporting. Works very well when the motion is predictable and there is no occlusion.
- **Cons:** Fails under large motion.

## GOTURN tracker

Out of all the tracking algorithms in the tracker class, this is the only one based on Convolutional Neural Network (CNN). It is also the only one that uses an offline trained model, because of which it is faster that other trackers. From OpenCV documentation, we know it is “robust to viewpoint changes, lighting changes, and deformations”. But it does not handle occlusion very well.

**Bug Alert:** There appears to be a bug and I cannot get this one to run in OpenCV 3.3.

In [4]:
# this is an array of the trackers available to us in OpenCV
cvTrackers = {
    'BOOSTING': cv2.TrackerBoosting_create,
    'MIL': cv2.TrackerMIL_create,
    'KCF': cv2.TrackerKCF_create,
    'TLD': cv2.TrackerTLD_create,
    'MEDIANFLOW': cv2.TrackerMedianFlow_create,
    'GOTURN': cv2.TrackerGOTURN_create  # this seems to be broken in python
}

In [7]:
# So let's load a simple video of Charlie Chaplin, we are going to view it first
IPython.display.Video('chaplin.mp4')

In [8]:
# load the input video
video = cv2.VideoCapture("chaplin.mp4")

pics = []

cnts = 0
while True:
    ok, frame = video.read()
    if not ok:
        print('Read {} frames'.format(cnts))
        break
    cnts += 1
    pics.append(frame)
video.release()

Read 150 frames


Now that we have the frames loaded into jupyter, we want to track his head. The following block of code iterates over 2 trackers (these seemed to produce the best performance for this application) and saves the output to an MPEG4 video. 

In [9]:
for tkr in ['KCF', 'MIL']:
    tracker = cvTrackers[tkr]()

    # Define an initial bounding box
    bbox = (287, 2, 77, 116)
    ok = tracker.init(pics[0], bbox)

    save_frames = []

    print('Running Tracker:', tkr)
    for i, frame in enumerate(pics):
        f = frame.copy()
        ok, bbox = tracker.update(f)
        # Draw bounding bxox
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(f, p1, p2, (255,0,0), 2)

            save_frames.append(f)
        else:
            print('Crap')
    videoWrite(save_frames, 'out_{}.mp4'.format(tkr))

Running Tracker: KCF
Running Tracker: MIL


In [10]:
%ls

OpenCV-Python-Tracking.ipynb  out_KCF.mp4
chaplin.mp4                   out_MIL.mp4


In [11]:
# lets watch the KCF video
IPython.display.Video('out_KCF.mp4')

In [22]:
# lets watch the MIL video
IPython.display.Video('out_MIL.mp4')


-----------

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.